In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import numpy as np
import neurospyke as ns
import h5py
from scipy.signal import savgol_filter, butter, lfilter, resample
from scipy.ndimage import gaussian_filter1d
import os
import preprocessing
import json
import utils

In [ ]:
f = open('./settings.json')
settings = json.load(f)
f.close()

sampling_frequency = settings['sampling_frequency']
sampling_time = 1 / sampling_frequency
resampling_frequency = settings['resampling_frequency']
resampling_time = 1 / resampling_frequency

frequency_ratio = sampling_frequency / resampling_frequency

signal_duration = settings['signal_duration']

group = settings['group']
subject = settings['subject']
conditions = settings['conditions']
areas = list(settings['areas'].keys())
areas_labels = [settings['areas'][area]['label'] for area in areas]
n_channels = settings['n_channels']
n_stimuli = settings['n_stimuli']

bin_duration = settings['IFR_bin_duration']
trial_duration = settings['trial_duration']
trial_duration_view = settings['trial_duration_view']

In [ ]:
# Parameters Setting
conditions_idx = 0
areas_idx = 0
channel = 0
channel_labels = ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015']
check_transient_idx = 14
subject_information = utils.get_subject_information(group, subject, conditions[conditions_idx], areas_labels[areas_idx], channel)

In [ ]:
# Stimulus Detection
raw_data_paths = utils.get_raw_data_paths(group, subject, conditions[conditions_idx], areas)

stimulus_idxs_matrix = preprocessing.get_stimulus_idxs_matrix(raw_data_paths, conditions[conditions_idx], settings)
stimulus_idxs = preprocessing.get_median_stimulus_idxs(stimulus_idxs_matrix, tolerance_duration=0.1, settings=settings)

In [ ]:
# Analysis on a single channel, given the stimulus_idxs computed above
f = h5py.File(os.path.join('./data', group, subject, conditions[conditions_idx], 'Raw_' + areas[areas_idx] +'_Ch_' + channel_labels[channel] + '.mat'))

raw_data = f['data']
raw_data = np.reshape(raw_data, np.size(raw_data))
raw_data = raw_data[np.arange(signal_duration * sampling_frequency)]

In [ ]:
# Stimulus Artifacts Suppression
data = np.copy(raw_data)

fig_data = ns.visualization.pyplot.figure(figsize=[6, 2])
gs = ns.visualization.pyplot.GridSpec(1, 2)
ax0 = fig_data.add_subplot(gs[0, 0])
ax1 = fig_data.add_subplot(gs[0, 1])
fig_data.suptitle(subject_information)

ns.visualization.plot_spikes(data, stimulus_idxs, ax=ax0, sampling_time=sampling_time, title='Raw Data')

stimulus_duration = 0.002 # seconds
stimulus_samples = int(np.round(stimulus_duration * sampling_frequency))
stimulus_half_samples = int(np.ceil(stimulus_samples / 2))
stimulus_range = np.arange(-stimulus_half_samples, stimulus_half_samples)

transient_duration = 0.75 # seconds
transient_samples = int(np.ceil(transient_duration * sampling_frequency))
transient_range = np.arange(stimulus_half_samples, stimulus_half_samples + transient_samples)

for idx in np.arange(np.size(stimulus_idxs)):
    stimulus_idx = stimulus_idxs[idx]
    data[stimulus_idx + stimulus_range] = 0
    transient = data[stimulus_idx + transient_range]

    max_idx = np.argmax(transient)
    # Ensure a meaningful max_idx, otherwhise set it null
    max_idx = max_idx[0] if type(max_idx) == np.ndarray else max_idx
    max_idx = 0 if max_idx < 100 else max_idx
    max_idx = np.size(transient) if np.abs(np.size(transient) - max_idx) < 100 else max_idx

    transient_smoothed = np.zeros(np.shape(transient))

    if max_idx != 0:
        left_transient_range = np.arange(stimulus_half_samples, stimulus_half_samples + max_idx) - stimulus_half_samples
        window_length =  int(np.floor(np.size(left_transient_range) / 5) * 2 + 1) # force odd length
        transient_smoothed[left_transient_range] = savgol_filter(transient[left_transient_range], window_length, 3)

    if max_idx != np.size(transient):
        right_transient_range = transient_range[max_idx:] - stimulus_half_samples
        window_length =  int(np.floor(np.size(right_transient_range) / 5) * 2 + 1) # force odd length
        transient_smoothed[right_transient_range] = savgol_filter(transient[right_transient_range], window_length, 3)

    transient_smoothed = savgol_filter(transient_smoothed, 501, 3) # remove left-right discontinuity

    data[stimulus_idx + transient_range] = data[stimulus_idx + transient_range] - transient_smoothed

    if idx == check_transient_idx:
        fig = ns.visualization.pyplot.figure()
        ax = fig.add_subplot(1, 1, 1)

        ns.visualization.plot_raw_data(transient, ax=ax, sampling_time=sampling_time)
        ns.visualization.plot_raw_data(transient_smoothed, ax=ax, sampling_time=sampling_time, color='#FF0000')
        ns.visualization.plot_raw_data(transient - transient_smoothed, ax=ax, sampling_time=sampling_time, color='#00FF00', title='Transient #' + str(idx))

        fig.suptitle(subject_information)

ns.visualization.plot_spikes(data, stimulus_idxs, ax=ax1, sampling_time=sampling_time, title='Data')
fig_data.set_tight_layout(True)

In [ ]:
num, den = butter(2, [300, 7000], btype='bandpass', fs=sampling_frequency)
filtered_data = lfilter(num, den, data)

ns.visualization.plot_spikes(filtered_data, stimulus_idxs, sampling_time=sampling_time, title='Filtered Data')
ax = ns.visualization.pyplot.gca()
ns.visualization.pyplot.text(0.5, 0.98, subject_information, fontsize=8, transform=ax.transAxes, horizontalalignment='center')

In [ ]:
from scipy.stats import median_abs_deviation
threshold = 5*1.4824*median_abs_deviation(filtered_data)
spikes_idxs, _ = ns.spikes.hard_threshold_local_maxima(filtered_data, threshold, 0.001, use_abs=True, sampling_time=sampling_time)

ns.visualization.plot_spikes(filtered_data, spikes_idxs, sampling_time=sampling_time, title='Spike Detection (HTLM)')
ax = ns.visualization.pyplot.gca()
ns.visualization.pyplot.text(0.5, 0.98, subject_information, fontsize=8, transform=ax.transAxes, horizontalalignment='center')

In [ ]:
ns.visualization.plot_spike_train([spikes_idxs, stimulus_idxs], channel_labels=['Spikes', 'Stimuli'], color=['#000000', '#FF0000'], linewidth=0.15, sampling_time=sampling_time, figsize=(9, 1), dpi=100, ylabel='')
spike_train = ns.utils.convert_spikes_idxs_to_spike_train(spikes_idxs, sampling_time, duration=signal_duration)
ax = ns.visualization.pyplot.gca()
ns.visualization.pyplot.text(0.5, 0.97, subject_information, fontsize=8, transform=ax.transAxes, horizontalalignment='center')

In [ ]:
# IFR Computation
IFR, bin_samples = preprocessing.get_IFR(spike_train, bin_duration, sampling_time)
ns.visualization.plot_spikes(IFR, stimulus_idxs, sampling_time=sampling_time, title='IFR', ylabel='Spikes/s')
ax = ns.visualization.pyplot.gca()
ns.visualization.pyplot.text(0.5, 0.98, subject_information, fontsize=8, transform=ax.transAxes, horizontalalignment='center')

In [ ]:
# IFR Filtering (Gaussian) and Resampling
filtered_IFR = gaussian_filter1d(IFR, 0.95 * bin_samples)

resampled_IFR = resample(filtered_IFR, int(np.floor(np.size(spike_train, axis=0) / frequency_ratio)))
resampled_stimulus_idxs = np.round(stimulus_idxs / frequency_ratio).astype(np.int_)
ns.visualization.plot_spikes(resampled_IFR, resampled_stimulus_idxs, sampling_time=resampling_time, title='IFR (filtered and resampled)', ylabel='Spikes/s')
ax = ns.visualization.pyplot.gca()
ns.visualization.pyplot.text(0.5, 0.98, subject_information, fontsize=8, transform=ax.transAxes, horizontalalignment='center')

In [ ]:
# IFR Trials Partition
IFR_trials = preprocessing.shape_trials(resampled_IFR, resampled_stimulus_idxs, trial_duration=trial_duration_view, sampling_time=resampling_time)

n_cols = 6
n_rows = int(np.ceil(n_stimuli / n_cols))
fig, axs = ns.visualization.pyplot.subplots(n_rows, n_cols, figsize=[12, 8], constrained_layout=True)

for trial_idx in np.arange(np.size(IFR_trials, 0)):
    row = int(np.floor(trial_idx / n_cols))
    col = trial_idx % n_cols

    IFR_trial = IFR_trials[trial_idx, :]
    
    ns.visualization.plot_raw_data(IFR_trial, ax=axs[row, col], sampling_time=resampling_time, title='#' + str(trial_idx + 1), ylabel='spikes/s')

fig.suptitle('IFR (all trials)\n' + subject_information, fontsize=16)

In [ ]:
# Spike Train Trials Partition
spike_train_trials = preprocessing.shape_trials(spike_train, stimulus_idxs, trial_duration=trial_duration_view, sampling_time=sampling_time)
ns.visualization.plot_raster(spike_train_trials, sampling_time, is_train=True, figsize=[3, 3])